In [14]:
import pandas as pd
from datetime import datetime, timedelta

In [2]:
df = pd.read_csv(
    '../postgis/data/gpx-route.csv', 
    header=None, 
    names=[
        'latitude',
        'longitude',
        'accumulated_distance',
        'estimated_time',
        'address',
    ],
)

In [3]:
df.head(3)

,latitude,longitude,accumulated_distance,estimated_time,address
0,1.34228,103.83519,0.000000,2024-05-17 21:00:00.000000,MacRitchie Nature Trail
1,1.34265,103.83574,0.073616,2024-05-17 21:00:00.000000,MacRitchie Nature Trail
2,1.34276,103.83592,0.097054,2024-05-17 21:01:00.983782,MacRitchie Nature Trail


In [25]:
# Calculate the number of full 30 km segments for each point
df['full_segments'] = df['accumulated_distance'] // 30

# Calculate the remaining distance for each point
df['remaining_distance'] = df['accumulated_distance'] % 30

# Calculate the total time for full segments for each point (6 hours per segment)
df['total_time_hours'] = df['full_segments'] * 6

# Calculate the pace for the remaining distance (at least 30 km in 6 hours)
df['remaining_pace_km_per_hour'] = 30 / 6

# Calculate the time for the remaining distance for each point
df['remaining_time_hours'] = df['remaining_distance'] / df['remaining_pace_km_per_hour']

# Calculate the total time to complete the journey for each point
df['total_time_hours'] += df['remaining_time_hours']

# How much total time changes
df['total_time_hours_delta'] = df['total_time_hours'].diff()

df.head(3)

,latitude,longitude,accumulated_distance,estimated_time,address,full_segments,remaining_distance,total_time_hours,remaining_pace_km_per_hour,remaining_time_hours,planned_time,total_time_hours_delta
0,1.34228,103.83519,0.000000,2024-05-17 21:00:00.000000,MacRitchie Nature Trail,0.0,0.000000,0.000000,5.0,0.000000,2024-05-17 21:00:00.000000,NaN
1,1.34265,103.83574,0.073616,2024-05-17 21:00:00.000000,MacRitchie Nature Trail,0.0,0.073616,0.014723,5.0,0.014723,2024-05-17 21:00:00.000000,0.014723
2,1.34276,103.83592,0.097054,2024-05-17 21:01:00.983782,MacRitchie Nature Trail,0.0,0.097054,0.019411,5.0,0.019411,2024-05-17 21:00:53.003570,0.004688


In [29]:
time_at_each_point = []

start_time = datetime(2024, 5, 17, 21, 0, 0) 
current_time = start_time

for i in range(len(df)):
    time_at_each_point.append(current_time)
    required_hours = df['total_time_hours_delta'].iloc[i]
    if not pd.isna(required_hours):
        current_time += timedelta(hours=required_hours)

df['planned_time'] = time_at_each_point

In [30]:
df.tail(3)

,latitude,longitude,accumulated_distance,estimated_time,address,full_segments,remaining_distance,total_time_hours,remaining_pace_km_per_hour,remaining_time_hours,planned_time,total_time_hours_delta
9205,1.341780,103.834430,321.490101,2024-05-20 22:58:19.088437,Reservoir Road,10.0,21.490101,64.298020,5.0,4.298020,2024-05-20 13:17:31.736112,0.005871
9206,1.341820,103.834530,321.502079,2024-05-20 22:58:43.407701,Reservoir Road,10.0,21.502079,64.300416,5.0,4.300416,2024-05-20 13:17:52.873007,0.002396
9207,1.342183,103.835156,321.582559,2024-05-20 22:58:53.330233,MacRitchie Nature Trail,10.0,21.582559,64.316512,5.0,4.316512,2024-05-20 13:18:01.497097,0.016096


In [31]:
df.to_csv(
    '../postgis/data/gpx-route.csv', 
    columns=[
        'latitude',
        'longitude',
        'accumulated_distance',
        'estimated_time',
        'address',
        'total_time_hours',
        'planned_time',
    ],
    header=False,
    index=False,
)